In [1]:
#pip install librosa
#!pip install tensorflow-gpu
#import librosa
import tensorflow
import IPython
import librosa

import numpy as np
#import scipy.io.wavfile as wave
import tensorflow as tf
import sys



C:\Users\56913\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\56913\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\56913\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\56913\AppData\Roaming\Python\Python36\site-

In [2]:
tf.reset_default_graph()
sys.path.append('./OpenSeq2Seq')
from open_seq2seq.utils.utils import deco_print, get_base_config, check_logdir,\
                                     create_logdir, create_model, get_interactive_infer_results


In [3]:


# Define the command line arguments that one would pass to run.py here
args_S2T = ["--config_file=OpenSeq2Seq/example_configs/speech2text/ds2_large_8gpus_mp.py",
        "--mode=interactive_infer",
        "--logdir=OpenSeq2Seq/ds2_large/",    
        "--batch_size_per_gpu=1",
        "--num_gpus=1"
]#"--batch_size_per_gpu=32",


# A simpler version of what run.py does. It returns the created model and its saved checkpoint
def get_model(args, scope):
    with tf.variable_scope(scope):
        args, base_config, base_model, config_module = get_base_config(args)
        checkpoint = check_logdir(args, base_config)
        model = create_model(args, base_config, config_module, base_model, None)
    return model, checkpoint

model_S2T, checkpoint_S2T = get_model(args_S2T, "S2T")


# Create the session and load the checkpoints
sess_config = tf.ConfigProto(allow_soft_placement=True)
sess_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=sess_config)
vars_S2T = {}
# vars_T2S = {}
for v in tf.get_collection(tf.GraphKeys.VARIABLES):
    if "S2T" in v.name:
        vars_S2T["/".join(v.op.name.split("/")[1:])] = v

saver_S2T = tf.train.Saver(vars_S2T)

saver_S2T.restore(sess, checkpoint_S2T)



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



*** Restoring from the latest checkpoint
*** Inference config:
{'batch_size_per_gpu': 1,
 'data_layer': <class 'open_seq2seq.data.speech2text.speech2text.Speech2TextDataLayer'>,
 'data_layer_params': {'dataset_files': [],
                       'input_type': 'spectrogram',
                       'num_audio_features': 160,
                       'shuffle': False,
                       'vocab_file': 'OpenSeq2Seq/open_seq2seq/test_utils/toy_speech_data/vocab.txt'},
 'decoder': <class 'open_seq2seq.decoders.fc_decoders.FullyConnectedCTCDecoder'>,
 'decoder_params': {'alpha': 2.0,
                    'alphabet_config_path': 'open

In [4]:
# Define the inference function

def infer(wave1_name,wave2_name):#
    
    wav1,sr1 = librosa.load(wave1_name)
    wav2,sr2 = librosa.load(wave2_name)
    print('first people')
    audio1 = IPython.display.Audio(wav1, rate = sr1)
    IPython.display.display(audio1)
    print('second people')
    audio2 = IPython.display.Audio(wav2, rate = sr2)
    IPython.display.display(audio2)
    
    wav1 = librosa.core.resample(wav1, sr1, 16000)
    wav2 = librosa.core.resample(wav2, sr2, 16000)

    

    # Recognize speech
    model_in1 = wav1
    model_in2 = wav2
    
    recognized1 = get_interactive_infer_results(model_S2T, sess, model_in=[model_in1])[0][0]
    recognized2 = get_interactive_infer_results(model_S2T, sess, model_in=[model_in2])[0][0]
    print("Recognized Speech")
    print(recognized1)
    print(recognized2)
    return recognized1, recognized2

In [5]:
# wav1_name = './TEST/DR1/FAKS0/SA1.WAV'
# wav2_name = './TEST/DR1/FAKS0/SA2.WAV'
# while True:
# file_dir = './TEST/DR1/FAKS0/'
#     print('input first audio')
#     wav1_name = input()
#     print('input second audio')
#     wav2_name = input()
#     if wav1_name == "" or wav2_name == "":
#         break

#     wav1_name = file_dir + wav1_name


#     wav2_name = file_dir + wav2_name

# IPython.display.clear_output()
# recognized1, recognized2 = infer(wav1_name,wav2_name)

In [5]:
sys.path.append('./Real-Time-Voice-Cloning')
from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import argparse
import torch
import IPython
import os
import sys
import re
import pandas as pd 


In [6]:
def load_model():
    #load arguments
#     tts_parser = argparse.ArgumentParser()#formatter_class = argparse.ArgumentDefaultsHelpFormatter
#     tts_parser.add_argument("--encoder",type = Path, default = 'Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt')
#     tts_parser.add_argument("--synthesizer",type = Path, default = "Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/")
#     tts_parser.add_argument("--vocoder",type = Path, default = "Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt")
    
#     args = tts_parser.parse_args()
    
    #cuda device
    device_id = torch.cuda.current_device()
    gpu_properties = torch.cuda.get_device_properties(device_id)
    print(gpu_properties)

    #load model
    #load encoder
    encoder.load_model(Path('Real-Time-Voice-Cloning/encoder/saved_models/pretrained.pt'))
    #load synthesizer
    synthesizer = Synthesizer(Path('Real-Time-Voice-Cloning/synthesizer/saved_models/logs-pretrained/taco_pretrained/'),low_mem = True)
    #vocoder
    vocoder.load_model(Path('Real-Time-Voice-Cloning/vocoder/saved_models/pretrained/pretrained.pt'))
    print('load model successfully')
    return encoder,synthesizer,vocoder

In [7]:
def load_audio(file_dir,voice1,voice2):
    in_fpath1 = Path((file_dir+voice1).replace("\"", "").replace("\'", ""))
    original_wave1, sampling_rate1 = librosa.load(in_fpath1)
    in_fpath2 = Path((file_dir+voice2).replace("\"", "").replace("\'", ""))
    original_wave2, sampling_rate2 = librosa.load(in_fpath2)
    IPython.display.display(IPython.display.Audio(original_wave1, rate = sampling_rate1))
    IPython.display.display(IPython.display.Audio(original_wave2, rate = sampling_rate2))
    print('success')
    return [original_wave1,sampling_rate1,original_wave2,sampling_rate2]

In [8]:
encoder,synthesizer,vocoder = load_model()

_CudaDeviceProperties(name='GeForce GTX 1050 Ti with Max-Q Design', major=6, minor=1, total_memory=4096MB, multi_processor_count=6)
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at Real-Time-Voice-Cloning\vocoder\saved_models\pretrained\pretrained.pt
load model successfully


In [9]:
# print('input the file direction')
# file_dir = input()
# print('input first audio')
# wav1_name = input()
# print('input second audio')
# wav2_name = input()
import pyaudio
import wave
import librosa
import IPython
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "first_speaker.wav"
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("first speaker recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()
wf = wave.open("first_speaker.wav", 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()


first speaker recording
* done recording


In [10]:
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "second_speaker.wav"
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("second speaker recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()
wf = wave.open("second_speaker.wav", 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

second speaker recording
* done recording


In [11]:
original_wave1,sampling_rate1,original_wave2,sampling_rate2 = load_audio('./','first_speaker.wav','second_speaker.wav')

success


In [12]:

preprocessed_wave1 = encoder.preprocess_wav(original_wave1,sampling_rate1)
preprocessed_wave2 = encoder.preprocess_wav(original_wave2, sampling_rate2)
print('preprocess the waveform')
#create embedding
embeds = [encoder.embed_utterance(preprocessed_wave1),encoder.embed_utterance(preprocessed_wave2)]
print('create the speaker embedding')
#synthesize the text

recognized1, recognized2 = infer('./' + 'first_speaker.wav','./' + 'second_speaker.wav')
texts = [recognized2,recognized1]

#
specs = synthesizer.synthesize_spectrograms(texts,embeds)


preprocess the waveform
create the speaker embedding
first people


second people


Recognized Speech
lef is heart and use hithen
i ell i arlec roke and we've got


In [ ]:
wrap1 = specs[0]
wrap2 = specs[1]
print('synthesize the text')

#vocoder
generated_wave1 = vocoder.infer_waveform(wrap1)
generated_wave2 = vocoder.infer_waveform(wrap2)

synthesize the text
{| ██████████░░░░░░ 33000/57600 | Batch Size: 6 | Gen Rate: 3.0kHz | }

In [14]:
IPython.display.display(IPython.display.Audio(generated_wave1, rate = synthesizer.sample_rate))
IPython.display.display(IPython.display.Audio(generated_wave2, rate = synthesizer.sample_rate))


In [ ]:
speaker1_name = (file_dir + wav1_name).split('/')[-2]
speaker2_name = (file_dir + wav2_name).split('/')[-2]

In [ ]:
os.mkdir('./swapping_result/%s&%s'%(speaker1_name,speaker2_name))


In [ ]:
store_path = Path('./swapping_result/%s&%s/%s_swapped.WAV' % (speaker1_name,speaker2_name,speaker1_name))
librosa.output.write_wav(store_path, generated_wave1.astype(np.float32), synthesizer.sample_rate)
store_path = Path('./swapping_result/%s&%s/%s_swapped.WAV' % (speaker1_name,speaker2_name,speaker2_name))
librosa.output.write_wav(store_path, generated_wave2.astype(np.float32), synthesizer.sample_rate)

In [ ]:
store_path = Path('./swapping_result/%s&%s/%s_original.WAV' % (speaker1_name,speaker2_name,speaker1_name))
librosa.output.write_wav(store_path, original_wave1.astype(np.float32), sampling_rate1)
store_path = Path('./swapping_result/%s&%s/%s_original.WAV' % (speaker1_name,speaker2_name,speaker2_name))
librosa.output.write_wav(store_path, original_wave2.astype(np.float32), sampling_rate2)

In [ ]:
recorded, r = librosa.load(WAVE_OUTPUT_FILENAME,sr=None)

In [ ]:
IPython.display.Audio(recorded, rate = r)



In [18]:
import python_speech_features as psf
!activate tensorflow-gpu

Could not find conda environment: tensorflow-gpu
You can list all discoverable environments with `conda info --envs`.



In [17]:

import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))
a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a + b))


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNCanonicalToParams' with these attrs.  Registered devices: [CPU], Registered kernels:
  <no registered kernels>

	 [[node S2T/S2T/ForwardPass/ds2_encoder/cudnn_gru/cudnn_gru/CudnnRNNCanonicalToParams (defined at C:\Users\56913\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\contrib\cudnn_rnn\python\ops\cudnn_rnn_ops.py:1251)  = CudnnRNNCanonicalToParams[T=DT_HALF, direction="bidirectional", dropout=0, input_mode="linear_input", num_params=60, rnn_mode="gru", seed=0, seed2=569, _device="/device:GPU:0"](S2T/S2T/ForwardPass/ds2_encoder/cudnn_gru/cudnn_gru/CudnnRNNCanonicalToParams/num_layers, S2T/S2T/ForwardPass/ds2_encoder/cudnn_gru/cudnn_gru/CudnnRNNCanonicalToParams/num_units, S2T/S2T/ForwardPass/ds2_encoder/cudnn_gru/cudnn_gru/CudnnRNNCanonicalToParams/input_size, S2T/S2T/ForwardPass/ds2_encoder/cudnn_gru/cudnn_gru/random_uniform, S2T/S2T/ForwardPass/ds2_encoder/cudnn_gru/cudnn_gru/random_uniform_1, 

In [19]:
import torch

In [21]:
torch.cuda.is_available()

True

In [22]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))